**MODEL 1**

In [1]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.
!pip install optuna


In [2]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta
import pathway as pw
import bokeh.plotting
import panel as pn
import optuna

In [3]:
import pandas as pd
import numpy as np
import optuna

# Load and preprocess dataset
df = pd.read_csv("dataset.csv")

df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
df['LotID_Encoded'], unique_lots = pd.factorize(df['SystemCodeNumber'])

df = df.sort_values(by=["LotID_Encoded", "Timestamp"]).reset_index(drop=True)
df = df[["Timestamp", "Occupancy", "Capacity", "LotID_Encoded"]]

# Function to simulate price with given alpha
def compute_price(alpha, base_price=10.0):
    return base_price + alpha * (df["Occupancy"] / (df["Capacity"] + 1e-6))

# Optuna objective
def objective(trial):
    alpha = trial.suggest_float("alpha", 0.0, 5.0)

    df["Price"] = compute_price(alpha=1)

    # Daily average grouping
    df["day"] = df["Timestamp"].dt.floor("D")
    grouped = df.groupby("day").agg({
        "Occupancy": "mean",
        "Price": "mean"
    }).dropna()

    if grouped["Occupancy"].std() == 0 or grouped["Price"].std() == 0:
        return float("inf")  # Avoid divide-by-zero

    corr = np.corrcoef(grouped["Occupancy"], grouped["Price"])[0, 1]
    return -corr  # Maximize correlation => minimize negative correlation

# Run Optuna
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction="minimize", sampler=sampler)

study.optimize(objective, n_trials=100)
# Final best alpha
print("\n✅ Best alpha found:")
print(f"alpha = {study.best_params['alpha']:.4f}")


[I 2025-07-10 04:56:04,142] A new study created in memory with name: no-name-6ee432a9-45aa-4db0-99d1-2741bc9454fa
[I 2025-07-10 04:56:04,159] Trial 0 finished with value: -0.9950815428200298 and parameters: {'alpha': 1.8727005942368125}. Best is trial 0 with value: -0.9950815428200298.
[I 2025-07-10 04:56:04,173] Trial 1 finished with value: -0.9950815428200298 and parameters: {'alpha': 4.75357153204958}. Best is trial 0 with value: -0.9950815428200298.
[I 2025-07-10 04:56:04,188] Trial 2 finished with value: -0.9950815428200298 and parameters: {'alpha': 3.6599697090570253}. Best is trial 0 with value: -0.9950815428200298.
[I 2025-07-10 04:56:04,212] Trial 3 finished with value: -0.9950815428200298 and parameters: {'alpha': 2.993292420985183}. Best is trial 0 with value: -0.9950815428200298.
[I 2025-07-10 04:56:04,247] Trial 4 finished with value: -0.9950815428200298 and parameters: {'alpha': 0.7800932022121826}. Best is trial 0 with value: -0.9950815428200298.
[I 2025-07-10 04:56:04,2


✅ Best alpha found:
alpha = 1.8727


In [11]:

pn.extension()

# --- Load and preprocess the data ---
df = pd.read_csv("dataset.csv")

# Timestamp parsing
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
df['LotID_Encoded'], unique_lots = pd.factorize(df['SystemCodeNumber'])

# Sort by lot + timestamp
df = df.sort_values(by=["LotID_Encoded", "Timestamp"]).reset_index(drop=True)

# --- Use tuned alpha ---
alpha = 1.8727   # Replaced with your Optuna-tuned value initially was 1
base_price = 10.0

# --- Apply pricing formula (pointwise, not recursive) ---
df["Price"] = base_price + alpha * (df["Occupancy"] / (df["Capacity"] + 1e-6))

# Optionally clamp price
df["Price"] = df["Price"].clip(lower=5, upper=20)

# Add 'day' column for windowing
df["day"] = df["Timestamp"].dt.floor("D")

# --- Daily Aggregation: mean price per day per lot ---
daily_df = df.groupby(["day", "LotID_Encoded"]).agg({
    "Price": "mean"
}).reset_index().rename(columns={"day": "Timestamp"})

# --- Plotting Function ---
def price_plotter(source, lot_id):
    fig = bokeh.plotting.figure(
        height=300,
        width=800,
        title=f"Daily Avg Price – LotID {lot_id}",
        x_axis_type="datetime"
    )
    fig.line("Timestamp", "Price", source=source, line_width=2, color="blue")
    fig.circle("Timestamp", "Price", source=source, size=5, color="black")
    return fig

# --- Create plots per lot ---
plots = []
for lot_id, lot_name in enumerate(unique_lots):
    lot_df = daily_df[daily_df["LotID_Encoded"] == lot_id]
    source = bokeh.models.ColumnDataSource(lot_df)
    fig = price_plotter(source, lot_name)
    plots.append(fig)

pn.Column(*plots).servable()


Column
    [0] Bokeh(figure)
    [1] Bokeh(figure)
    [2] Bokeh(figure)
    [3] Bokeh(figure)
    [4] Bokeh(figure)
    [5] Bokeh(figure)
    [6] Bokeh(figure)
    [7] Bokeh(figure)
    [8] Bokeh(figure)
    [9] Bokeh(figure)
    [10] Bokeh(figure)
    [11] Bokeh(figure)
    [12] Bokeh(figure)
    [13] Bokeh(figure)

**MODEL 2**

In [12]:
import pandas as pd
import numpy as np
import optuna

# Step 1: Load the dataset
df = pd.read_csv("dataset.csv")

# Step 2: Preprocessing
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                 format='%d-%m-%Y %H:%M:%S')

df['LotID_Encoded'], unique_lots = pd.factorize(df['SystemCodeNumber'])

# Encode TrafficConditionNearby
traffic_map = {'low': 1, 'average': 2, 'high': 3}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map).fillna(2)

# Encode VehicleType
vehicle_map = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_map).fillna(1.0)

# Step 3: Define the objective function for Optuna
def objective(trial):
    α = trial.suggest_float("α", 0.0, 5.0)
    β = trial.suggest_float("β", 0.0, 5.0)
    γ = trial.suggest_float("γ", 0.0, 5.0)
    δ = trial.suggest_float("δ", 0.0, 5.0)
    ε = trial.suggest_float("ε", 0.0, 5.0)

    # Calculate RawDemand
    df["RawDemand"] = (
        α * (df["Occupancy"] / (df["Capacity"] + 1e-6)) +
        β * df["QueueLength"] -
        γ * df["TrafficLevel"] +
        δ * df["IsSpecialDay"] +
        ε * df["VehicleWeight"]
    )

    # Calculate correlation with Occupancy as the proxy for effectiveness
    if df["RawDemand"].std() == 0:
        return float("inf")  # avoid NaN correlation

    corr = np.corrcoef(df["RawDemand"], df["Occupancy"])[0, 1]

    return -corr  # We want to maximize correlation → minimize -corr

# Step 4: Run the Optuna study
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction="minimize", sampler=sampler)

study.optimize(objective, n_trials=100)
# Step 5: Print the best parameters
print("\n✅ Best Parameters Found:")
for param, value in study.best_params.items():
    print(f"{param} = {value:.4f}")


[I 2025-07-10 04:58:59,030] A new study created in memory with name: no-name-d5db8647-42d9-4592-9095-6bb66ee2a97d
[I 2025-07-10 04:58:59,047] Trial 0 finished with value: -0.3292350772871832 and parameters: {'α': 1.8727005942368125, 'β': 4.75357153204958, 'γ': 3.6599697090570253, 'δ': 2.993292420985183, 'ε': 0.7800932022121826}. Best is trial 0 with value: -0.3292350772871832.
[I 2025-07-10 04:58:59,052] Trial 1 finished with value: 0.11122426456899354 and parameters: {'α': 0.7799726016810132, 'β': 0.2904180608409973, 'γ': 4.330880728874676, 'δ': 3.005575058716044, 'ε': 3.540362888980227}. Best is trial 0 with value: -0.3292350772871832.
[I 2025-07-10 04:58:59,059] Trial 2 finished with value: -0.3409670190280264 and parameters: {'α': 0.10292247147901223, 'β': 4.8495492608099715, 'γ': 4.162213204002109, 'δ': 1.0616955533913808, 'ε': 0.9091248360355031}. Best is trial 2 with value: -0.3409670190280264.
[I 2025-07-10 04:58:59,078] Trial 3 finished with value: -0.31876722238952576 and par


✅ Best Parameters Found:
α = 4.5762
β = 1.5186
γ = 3.5736
δ = 0.0113
ε = 1.5615


In [13]:
# --- Preprocessing + Stream Setup ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
import pathway as pw
import bokeh.plotting
import panel as pn

# -----------------------------
# Pandas Preprocessing
# -----------------------------
df = pd.read_csv('dataset.csv')

# Timestamp
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                 format='%d-%m-%Y %H:%M:%S')

# Encode Lot IDs
df['LotID_Encoded'], unique_lots = pd.factorize(df['SystemCodeNumber'])

# Traffic encoding
traffic_map = {'low': 1, 'average': 2, 'high': 3}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map).fillna(2)

# Vehicle encoding
vehicle_map = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_map).fillna(1.0)

# Sort by time
df = df.sort_values(by='Timestamp').reset_index(drop=True)

# Save minimal CSV for Pathway stream
df[[
    "Timestamp", "Occupancy", "Capacity", "QueueLength",
    "TrafficLevel", "IsSpecialDay", "VehicleWeight", "LotID_Encoded"
]].to_csv("parking_stream_model2.csv", index=False)

# -----------------------------
# Pathway Streaming
# -----------------------------
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficLevel: int
    IsSpecialDay: int
    VehicleWeight: float
    LotID_Encoded: int

# Load the stream
data = pw.demo.replay_csv("parking_stream_model2.csv", schema=ParkingSchema, input_rate=1000)

# Parse timestamp and day
fmt = "%Y-%m-%d %H:%M:%S"
data = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# Model 2 demand parameters
# ✅ Best Parameters Found:
# α = 4.5762
# β = 1.5186
# γ = 3.5736
# δ = 0.0113
# ε = 1.5615
α, β, γ, δ, ε = 4.5762, 1.5186, 3.5736, 0.0113, 1.5615
base_price = 10.0
λ = 1.0

# Calculate demand
data = data.with_columns(
    RawDemand = α * (data.Occupancy / data.Capacity) +
                β * data.QueueLength -
                γ * data.TrafficLevel +
                δ * data.IsSpecialDay +
                ε * data.VehicleWeight
)

# 1-day tumbling delta window
windowed = data.windowby(
    pw.this.t,
    instance=(pw.this.day, pw.this.LotID_Encoded),
    window=pw.temporal.tumbling(timedelta(days=1)),
    behavior=pw.temporal.exactly_once_behavior()
).reduce(
    t = pw.this._pw_window_end,
    LotID_Encoded = pw.reducers.max(pw.this.LotID_Encoded),
    sum_demand = pw.reducers.sum(pw.this.RawDemand),
    count_demand = pw.reducers.count(),
    demand_min = pw.reducers.min(pw.this.RawDemand),
    demand_max = pw.reducers.max(pw.this.RawDemand)
)

# Step 1: Compute average demand
windowed = windowed.with_columns(
    demand_avg = pw.this.sum_demand / (pw.this.count_demand + 1e-6)
)

# Step 2: Normalize demand within window
windowed = windowed.with_columns(
    NormalizedDemand = (pw.this.demand_avg - pw.this.demand_min) /
                       (pw.this.demand_max - pw.this.demand_min + 1e-6)
)

# Step 3: Compute unclamped and clamped price using pw.if_else
# Step 1: Compute unclamped price
windowed = windowed.with_columns(
    unclamped_price = base_price * (1 + λ * pw.this.NormalizedDemand)
)

# Step 2: Clamp the price
windowed = windowed.with_columns(
    price = pw.if_else(
        pw.this.unclamped_price > 2 * base_price,
        2 * base_price,
        pw.if_else(
            pw.this.unclamped_price < 0.5 * base_price,
            0.5 * base_price,
            pw.this.unclamped_price
        )
    )
)

# -----------------------------
# Plotting
# -----------------------------
pn.extension()

def price_plotter(source, lot_id):
    fig = bokeh.plotting.figure(
        height=300,
        width=800,
        title=f"Model 2 Price Evolution – LotID {lot_id}",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.circle("t", "price", source=source, size=5, color="red")
    return fig

# Plot for all lots
plots = []
for lot_id, lot_name in enumerate(unique_lots):
    lot_table = windowed.filter(pw.this.LotID_Encoded == lot_id)
    viz = lot_table.plot(lambda source, name=lot_name: price_plotter(source, name), sorting_col="t")
    plots.append(viz)

pn.Column(*plots).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [1] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [2] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [3] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [4] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [5] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [6] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [7] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [8] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [9] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [10] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [11] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [12] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [13] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [16]:
pw.run()

Output()

In [14]:
import pandas as pd
import numpy as np

# --------------------------
# Load and preprocess data
df = pd.read_csv("dataset.csv")

df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
df['LotID_Encoded'], unique_lots = pd.factorize(df['SystemCodeNumber'])
df["day"] = df["Timestamp"].dt.floor("D")
df = df.sort_values(by=["LotID_Encoded", "Timestamp"]).reset_index(drop=True)

# --------------------------
# MODEL 1: Daily Average Price
alpha_1 = 1.8727
base_price = 10.0
df["Model1_Price"] = base_price + alpha_1 * (df["Occupancy"] / (df["Capacity"] + 1e-6))
df["Model1_Price"] = df["Model1_Price"].clip(lower=5, upper=20)

model1_df = df.groupby(["day", "LotID_Encoded"]).agg({
    "Model1_Price": "mean"
}).reset_index().rename(columns={"day": "Timestamp"})

model1_df.to_csv("model1_daily.csv", index=False)

# --------------------------
# MODEL 2: Demand-Based Price
# Encode features if not done yet
traffic_map = {'low': 1, 'average': 2, 'high': 3}
vehicle_map = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
df["TrafficLevel"] = df["TrafficConditionNearby"].map(traffic_map).fillna(2)
df["VehicleWeight"] = df["VehicleType"].map(vehicle_map).fillna(1.0)

α, β, γ, δ, ε = 4.5762, 1.5186, 3.5736, 0.0113, 1.5615
λ = 1.0

df["RawDemand"] = (
    α * (df["Occupancy"] / (df["Capacity"] + 1e-6)) +
    β * df["QueueLength"] -
    γ * df["TrafficLevel"] +
    δ * df["IsSpecialDay"] +
    ε * df["VehicleWeight"]
)

# Aggregate RawDemand by day and lot
agg_demand = df.groupby(["day", "LotID_Encoded"]).agg({
    "RawDemand": ["sum", "count", "min", "max"]
})
agg_demand.columns = ["sum_demand", "count", "min_demand", "max_demand"]
agg_demand = agg_demand.reset_index().rename(columns={"day": "Timestamp"})

# Compute normalized demand
agg_demand["avg_demand"] = agg_demand["sum_demand"] / (agg_demand["count"] + 1e-6)
agg_demand["NormalizedDemand"] = (
    (agg_demand["avg_demand"] - agg_demand["min_demand"]) /
    (agg_demand["max_demand"] - agg_demand["min_demand"] + 1e-6)
)

# Compute Model 2 price
agg_demand["unclamped_price"] = base_price * (1 + λ * agg_demand["NormalizedDemand"])
agg_demand["Model2_Price"] = agg_demand["unclamped_price"].clip(lower=5, upper=20)

model2_df = agg_demand[["Timestamp", "LotID_Encoded", "Model2_Price"]]
model2_df.to_csv("model2_daily.csv", index=False)


In [15]:
import pandas as pd
import panel as pn
import bokeh.plotting
from bokeh.models import ColumnDataSource

pn.extension()

# Step 1: Load original dataset to recover lot names
df_orig = pd.read_csv("dataset.csv")
df_orig["LotID_Encoded"], lot_names = pd.factorize(df_orig["SystemCodeNumber"])
lot_id_to_name = dict(enumerate(lot_names))  # Mapping: 0 → 'BHMXXX...'

# Step 2: Load both model predictions
model1_df = pd.read_csv("model1_daily.csv")
model2_df = pd.read_csv("model2_daily.csv")

# Convert Timestamp to datetime
model1_df["Timestamp"] = pd.to_datetime(model1_df["Timestamp"])
model2_df["Timestamp"] = pd.to_datetime(model2_df["Timestamp"])

# Merge predictions
merged_df = pd.merge(
    model1_df, model2_df,
    on=["Timestamp", "LotID_Encoded"],
    how="inner"
)

# Step 3: Plotting function using actual names
def plot_model_comparison(df, lot_id):
    lot_name = lot_id_to_name.get(lot_id, f"Lot {lot_id}")
    lot_df = df[df["LotID_Encoded"] == lot_id]
    source = ColumnDataSource(lot_df)

    fig = bokeh.plotting.figure(
        height=300,
        width=800,
        title=f"Price Comparison – {lot_name}",
        x_axis_type="datetime"
    )

    fig.line("Timestamp", "Model1_Price", source=source, line_width=2, color="green", legend_label="Model 1")
    fig.line("Timestamp", "Model2_Price", source=source, line_width=2, color="blue", legend_label="Model 2")
    fig.circle("Timestamp", "Model1_Price", source=source, size=4, color="green")
    fig.circle("Timestamp", "Model2_Price", source=source, size=4, color="blue")

    fig.legend.location = "top_left"
    fig.legend.click_policy = "hide"

    return fig

# Step 4: Generate plots for all lot IDs
plots = []
for lot_id in merged_df["LotID_Encoded"].unique():
    fig = plot_model_comparison(merged_df, lot_id)
    plots.append(fig)

# Display
pn.Column(*plots).servable()


Column
    [0] Bokeh(figure)
    [1] Bokeh(figure)
    [2] Bokeh(figure)
    [3] Bokeh(figure)
    [4] Bokeh(figure)
    [5] Bokeh(figure)
    [6] Bokeh(figure)
    [7] Bokeh(figure)
    [8] Bokeh(figure)
    [9] Bokeh(figure)
    [10] Bokeh(figure)
    [11] Bokeh(figure)
    [12] Bokeh(figure)
    [13] Bokeh(figure)